# COVID-19 cases

Data comes from the ECDC.

Links 

* https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide
* https://www.kaggle.com/pavlofesenko/interactive-titanic-dashboard-using-bokeh
* https://stackoverflow.com/questions/41382310/adding-a-second-y-axis-in-bokehjs

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import pylab as plt
pd.set_option('display.width',140)
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, ColorBar, HoverTool, Legend
from bokeh.plotting import figure
from bokeh.palettes import brewer
from bokeh.layouts import row, column, gridplot
from bokeh.models import CustomJS, Slider, Select,MultiSelect, Plot, Button, LinearAxis, Range1d, DatetimeTickFormatter
from bokeh.models.glyphs import Line, MultiLine
from bokeh.palettes import Category10
output_notebook()
#output_file('test.html')

Loading BokehJS ...

In [12]:
#df = pd.read_csv('ecdc_data.csv')
df = pd.read_excel('https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide.xlsx')
df['dateRep'] = pd.to_datetime(df.dateRep, infer_datetime_format=True)
df = df.sort_values(['countriesAndTerritories','dateRep'])
#find cumulative cases in each country by using groupby-apply
df['cumcases'] = df.groupby(['countriesAndTerritories'])['cases'].apply(lambda x: x.cumsum())
data = pd.pivot_table(df,index='dateRep',columns='countriesAndTerritories',values='cumcases').reset_index()
#data = data.fillna(0)
summary = df.groupby('countriesAndTerritories').agg({'deaths':np.sum,'cases':np.sum}).reset_index()

In [13]:
summary

,countriesAndTerritories,deaths,cases
0,Afghanistan,4,114
1,Albania,10,212
2,Algeria,29,454
3,Andorra,6,334
4,Angola,2,7
...,...,...,...
191,Uzbekistan,1,145
192,Venezuela,1,119
193,Vietnam,0,228
194,Zambia,0,29


In [37]:
data[30:35][['dateRep','China','Sweden','India']]

countriesAndTerritories,dateRep,China,Sweden,India
30,2020-01-30,7734.0,0.0,1.0
31,2020-01-31,9714.0,0.0,1.0
32,2020-02-01,11809.0,1.0,1.0
33,2020-02-02,14399.0,1.0,2.0
34,2020-02-03,17211.0,1.0,2.0


## bokeh plot with python callback

In [ ]:
def bokeh_plot_cases(attr, old, new):
    """Plot cases per country"""
    
    #countries = ['Ireland','Sweden']
    countries = select.value
    print (countries)
    p = figure(plot_width=500,plot_height=300,x_axis_type='datetime',tools="hover,xwheel_zoom,xpan")        
    source = ColumnDataSource(data)
    colors = Category10[10] + Category10[10]
    i=0
    items=[]
    for c in countries:
        line = Line(x='dateRep',y=c, line_color=colors[i],line_width=3,name=c)
        glyph = p.add_glyph(source, line)
        i+=1
        items.append((c,[glyph]))
    p.add_layout(Legend(
            location="top_left",
            items=items))        
    p.grid.visible = False
    return p

names = list(df.countriesAndTerritories.unique() )
select = MultiSelect(title="Option:", value=[names[0]], options=names)
select.on_change('value', bokeh_plot_cases)
plot = bokeh_plot_cases(None,None,None) 
g = gridplot([[select],[plot]], toolbar_location='below')
show(g)

## with javascript callbacks

In [18]:
source = ColumnDataSource(data)

#print (filt_data)
# create CDS for filtered sources
filt_data1 = data[['dateRep','China']].rename(columns={'China':'cases'})
src2 = ColumnDataSource(filt_data1)
filt_data2 = data[['dateRep','Ireland']].rename(columns={'Ireland':'cases'})
src3 = ColumnDataSource(filt_data2)

source.add(data['dateRep'].apply(lambda d: d.strftime('%Y-%m-%d')), 'date_formatted')

hover_tool = HoverTool(tooltips=[
            ('Cases', '@cases')]   
        )

p1 = figure(plot_width=600,plot_height=400,x_axis_type='datetime',
           tools=[hover_tool],title='Sars-CoV-2 cases (ECDC)',y_range=Range1d(start=0, end=filt_data1.cases.max()+50))
p1.line(x='dateRep',y='cases', source=src2, legend_label="country 1", line_color='blue',
        line_width=3,line_alpha=.8)
#set the second y-axis and use that with our second line
p1.extra_y_ranges = {"y2": Range1d(start=0, end=filt_data2.cases.max()+50)}
p1.add_layout(LinearAxis(y_range_name="y2"), 'right')
p1.line(x='dateRep',y='cases', source=src3, legend_label="country 2", line_color='orange',
        line_width=3,line_alpha=.8,y_range_name="y2")

p1.yaxis[0].axis_label = 'China'
p1.yaxis[1].axis_label = 'Ireland'
p1.background_fill_color = "whitesmoke"
p1.background_fill_alpha = 0.5
p1.legend.location = "top_left"
p1.xaxis.axis_label = 'Date'
p1.xaxis.formatter=DatetimeTickFormatter(days="%d/%m",
months="%m/%d %H:%M",
)
#this javascript snippet is the callback when either select is changed
code="""
var c = cb_obj.value;
ax.axis_label = c;
var y = s1.data[c];
s2.data['cases'] = y;
y_range.start = 0;
y_range.end = parseInt(y[y.length - 1]+50);
s2.change.emit();
"""
callback1 = CustomJS(args=dict(s1=source,s2=src2,y_range=p1.y_range,ax=p1.yaxis[0]), code=code)
callback2 = CustomJS(args=dict(s1=source,s2=src3,y_range=p1.extra_y_ranges['y2'],ax=p1.yaxis[1]), code=code)
names = list(df.countriesAndTerritories.unique() )
names_sub=['China','United_Kingdom','United_States_of_America','Spain','Italy',
           'Germany','France','Iran','Australia','Ireland','Sweden','Belgium','Turkey','India']
select1 = Select(title="Country 1:", value='China', options=names_sub)
select1.js_on_change('value', callback1)
select2 = Select(title="Country 2:", value='Ireland', options=names)
select2.js_on_change('value', callback2)
btn = Button(label='Update')

layout = column(row(select1,select2), row(p1))
show(layout)


## save the html

In [17]:
from bokeh.resources import CDN
from bokeh.embed import file_html

#plot = figure(plot_width=200)
#plot.circle([1,2], [3,4])

html = file_html(layout, CDN, "COVID-19 plots")
outfile = open("covid_plots.html",'w')
outfile.write(html)
outfile.close()